# Treasure Hunt Game Notebook

## Read and Review Your Starter Code
The theme of this project is a popular treasure hunt game in which the player needs to find the treasure before the pirate does. While you will not be developing the entire game, you will write the part of the game that represents the intelligent agent, which is a pirate in this case. The pirate will try to find the optimal path to the treasure using deep Q-learning. 

You have been provided with two Python classes and this notebook to help you with this assignment. The first class, TreasureMaze.py, represents the environment, which includes a maze object defined as a matrix. The second class, GameExperience.py, stores the episodes – that is, all the states that come in between the initial state and the terminal state. This is later used by the agent for learning by experience, called "exploration". This notebook shows how to play a game. Your task is to complete the deep Q-learning implementation for which a skeleton implementation has been provided. The code blocks you will need to complete has #TODO as a header.

First, read and review the next few code and instruction blocks to understand the code that you have been given.

### Setup and Imports
This section imports all required Python libraries and project modules.  
- **NumPy** for numerical operations  
- **Matplotlib** for plotting and visualization  
- **TensorFlow/Keras** for building and training the neural network  
- **TreasureMaze** and **GameExperience** are provided project files to define the environment and replay buffer.


In [ ]:
# Python 3 already has print() — this future import isn’t needed, but harmless
# from __future__ import print_function

import os, sys, time, datetime, json, random
import numpy as np
import matplotlib.pyplot as plt

# Use tensorflow.keras consistently
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, PReLU
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

# If you use the functional API elsewhere, you can also import:
# from tensorflow.keras import layers, models, optimizers

from TreasureMaze import TreasureMaze
from GameExperience import GameExperience

%matplotlib inline


### Model Resume Function
This helper function reloads the latest checkpointed model if one exists, or builds a fresh model otherwise.  
It allows training to continue across multiple sessions without losing progress.


In [ ]:
from tensorflow.keras.models import load_model

def resume_model():
    """
    Load the latest checkpoint if it exists, else build a fresh model.
    Returns (model, start_epoch).
    """
    ckpt, start_epoch = latest_checkpoint()
    if ckpt:
        model = load_model(ckpt)
        print(f"Resuming from {ckpt} (epoch {start_epoch})")
    else:
        model = build_model(qmaze)
        start_epoch = 0
        print("Starting fresh with new model")
    return model, start_epoch


### Directory Setup and Logging
Creates folders for saving model checkpoints and logs.  
Also defines a simple logging function (`log`) to keep training output consistent and easy to read.


In [ ]:
# Folders (ok if they already exist)
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("logs", exist_ok=True)

def log(msg):
    print(msg)


### Maze Definition
Defines the maze as a NumPy array, initializes the `TreasureMaze` environment,  
and prints the treasure target location along with an example valid starting cell.


In [ ]:
maze = np.array([
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.]
])

qmaze = TreasureMaze(maze)

print("Target:", qmaze.target)
print("Random valid start example:", random.choice(qmaze.free_cells))

### Maze Visualization
Provides a function to display the maze grid using matplotlib.  
Visited cells are shaded, the pirate’s current location is marked, and the treasure is highlighted.


In [ ]:
def show(qmaze):
    plt.grid('on')
    nrows, ncols = qmaze.maze.shape
    ax = plt.gca()
    ax.set_xticks(np.arange(0.5, nrows, 1))
    ax.set_yticks(np.arange(0.5, ncols, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    canvas = np.copy(qmaze.maze)
    for row, col in qmaze.visited:
        canvas[row, col] = 0.6
    pirate_row, pirate_col, _ = qmaze.state
    canvas[pirate_row, pirate_col] = 0.3     # pirate
    canvas[nrows-1, ncols-1] = 0.9           # treasure
    img = plt.imshow(canvas, interpolation='none', cmap='gray')
    return img



### Actions and Constants
Defines the four movement actions (`LEFT, UP, RIGHT, DOWN`) and assigns them integer values.  
Also creates a dictionary for human-readable labels and sets the initial exploration factor (`epsilon`).


This helper function allows a visual representation of the maze object:

In [ ]:
LEFT, UP, RIGHT, DOWN = 0, 1, 2, 3
actions_dict = {LEFT:'left', UP:'up', RIGHT:'right', DOWN:'down'}
epsilon = 0.1


### Utility Function
Ensures that input state data is reshaped into the correct batch format for the neural network’s `predict` function.


In [ ]:
import numpy as np

def as_batch(x):
    import numpy as np
    x = np.asarray(x)
    return x if x.ndim == 2 else x.reshape(1, -1)


### Neural Network Model
Defines a Deep Q-Network (DQN) with two hidden layers (64 neurons each).  
The model predicts Q-values for each possible action given the maze state.


In [ ]:
def build_model(qmaze):
    state_dim = qmaze.maze.size
    n_actions = 4

    model = Sequential()
    model.add(Dense(64, input_dim=state_dim, activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(n_actions, activation="linear"))
    model.compile(optimizer=Adam(learning_rate=1e-3), loss="mse")
    return model


### Gameplay Functions
- **play_game**: Runs a full game episode starting from a given cell using the current model. Returns the outcome and path.  
- **completion_check**: Verifies if the trained agent can solve the maze consistently from all valid starting positions.


In [ ]:
def play_game(model, qmaze, pirate_cell):
    qmaze.reset(pirate_cell)
    envstate = qmaze.observe()
    path = [pirate_cell]
    while True:
        q = model.predict(as_batch(envstate), verbose=0)[0]

        action = int(np.argmax(q))
        envstate, reward, game_status = qmaze.act(action)
        pr, pc, _ = qmaze.state
        path.append((pr, pc))
        if game_status == 'win':
            return True, path
        elif game_status == 'lose':
            return False, path

def completion_check(model, qmaze):
    for cell in qmaze.free_cells:
        if not qmaze.valid_actions(cell):
            return False
        won, _ = play_game(model, qmaze, cell)
        if not won:
            return False
    return True


### Evaluation Function
Runs multiple episodes from random starting positions and calculates the average win rate of the agent.



In [ ]:
def evaluate(model, qmaze, n_episodes=100):
    wins = 0
    for _ in range(n_episodes):
        start = random.choice(qmaze.free_cells)
        won, _ = play_game(model, qmaze, pirate_cell=start)
        if won:
            wins += 1
    return wins / float(n_episodes)


### Training Function (Deep Q-Learning)
Implements the reinforcement learning loop using:  
- Experience Replay (via `GameExperience`)  
- ε-greedy policy with decay for exploration vs. exploitation  
- Neural network Q-value updates using the Bellman equation  
- Logging and checkpoint saving for progress tracking  
- Optional early stopping if the agent reaches consistent perfect performance.


In [ ]:
def qtrain(model, qmaze, *,
           n_epoch_total=10000,
           start_epoch=0,
           max_memory=4000,
           batch_size=16,
           target_update_steps=500,    # kept for compatibility (not used with GameExperience)
           checkpoint_every=500,
           max_steps_per_episode=200,
           log_every=200):
    """
    Training loop using the provided GameExperience replay buffer.
    Expects:
      - GameExperience(model, max_memory=...) with .remember([s,a,r,s',done]) and .get_data(batch_size)
      - qmaze.reset(start_cell), qmaze.observe(), qmaze.act(action) -> (next_state, reward, status)
      - status is 'win' or 'lose' when terminal
      - as_batch(x) utility available to ensure (batch, features) input to model.predict(...)
      - log(msg) available for progress messages
    """

    import datetime
    import numpy as np
    import random

    # Replay buffer using your existing class
    experience = GameExperience(model, max_memory=max_memory)

    # Simple epsilon (exploration) schedule
    eps_start = 1.0 if start_epoch == 0 else 0.2
    eps_min   = 0.05
    eps_decay = 0.997
    epsilon   = max(eps_min, eps_start * (eps_decay ** start_epoch))

    # Rolling window to estimate recent win-rate
    win_history = []
    start_time = datetime.datetime.now()

    def fmt_time(seconds):
        s = int(seconds)
        if s < 60: return f"{s}s"
        m, s = divmod(s, 60)
        if m < 60: return f"{m}m{s:02d}s"
        h, m = divmod(m, 60); return f"{h}h{m:02d}m{s:02d}s"

    for ep in range(start_epoch, n_epoch_total):
        # random valid start each episode
        start_cell = random.choice(qmaze.free_cells)
        qmaze.reset(start_cell)
        envstate = qmaze.observe()

        steps = 0
        game_over = False
        loss = 0.0
        last_reward = 0.0
        status = None

        while not game_over and steps < max_steps_per_episode:
            steps += 1

            # ε-greedy policy
            if np.random.rand() < epsilon:
                action = random.choice([0, 1, 2, 3])  # LEFT, UP, RIGHT, DOWN
            else:
                q_vals = model.predict(as_batch(envstate), verbose=0)[0]
                action = int(np.argmax(q_vals))

            prev_envstate = envstate
            envstate, reward, status = qmaze.act(action)
            done = (status in ('win', 'lose'))
            last_reward = reward

            # store transition and train on a mini-batch
            experience.remember([prev_envstate, action, reward, envstate, done])
            X, y = experience.get_data(batch_size)
            if X is not None and y is not None and len(X) > 0:
                loss = float(model.train_on_batch(X, y))

            game_over = done

        # update epsilon (decay) at end of episode
        epsilon = max(eps_min, epsilon * eps_decay)

        # record win/lose
        win_history.append(1 if status == 'win' else 0)
        wr_window = win_history[-min(200, len(win_history)):]  # last up to 200 episodes
        wr200 = np.mean(wr_window) if wr_window else 0.0

        # log progress
        if (ep - start_epoch) % log_every == 0:
            dt = (datetime.datetime.now() - start_time).total_seconds()
            log(f"epoch {ep:05d} | eps={epsilon:.3f} | loss={loss:.4f} | steps={steps} | "
                f"reward={last_reward:.2f} | wr200={wr200:.2%} | time={fmt_time(dt)}")

        # checkpoint
        if (ep - start_epoch) % checkpoint_every == 0 and ep > start_epoch:
            save_path = f"checkpoints/pirate_epoch{ep}.h5"
            model.save(save_path)
            log(f"[ckpt] saved -> {save_path}")

        # optional: early stop if perfect in the window
        if len(wr_window) == 200 and sum(wr_window) == 200:
            # (optional) double-check full completion if you want:
            # if completion_check(model, qmaze): 
            save_path = f"checkpoints/pirate_epoch{ep}.h5"
            model.save(save_path)
            log(f"[early-stop] 100% wr200 at epoch {ep}. Saved -> {save_path}")
            break

    # final save at the target epoch
    save_path = f"checkpoints/pirate_epoch{max(ep, start_epoch)}.h5"
    model.save(save_path)
    log(f"[train] finished -> {save_path}")
    return model


### Checkpoint Management
Utility function to locate the most recent saved model checkpoint.  
This enables resuming training from the latest saved epoch instead of starting over.


In [ ]:
import glob, re

def latest_checkpoint():
    ckpts = glob.glob("checkpoints/pirate_epoch*.h5")
    if not ckpts:
        return None, 0
    latest = max(ckpts, key=lambda p: int(re.search(r'epoch(\d+)', p).group(1)))
    start_epoch = int(re.search(r'epoch(\d+)', latest).group(1))
    return latest, start_epoch


In [ ]:
def train_to(total_epochs):
    """
    Reload latest checkpoint (or build fresh), then train until total_epochs.
    Use in chunks: 2000 -> 5000 -> 15000, etc.
    """
    ckpt, start_epoch = latest_checkpoint()
    if ckpt:
        model = load_model(ckpt)
        log(f"Resuming from {ckpt} (epoch {start_epoch})")
    else:
        model = build_model(qmaze)
        start_epoch = 0
        log("Starting fresh")

    if start_epoch >= total_epochs:
        log(f"Already at {start_epoch} >= target {total_epochs}; skipping.")
        return model, start_epoch

    model = qtrain(
        model, qmaze,
        n_epoch_total=total_epochs,
        start_epoch=start_epoch,
        max_memory=2000,
        batch_size=16,
        target_update_steps=500,
        checkpoint_every=200,
        max_steps_per_episode=150,
        log_every=100
    )
    return model, total_epochs


### Training in Chunks (Example Run to 2000 Epochs)
Here we demonstrate how to train the agent in smaller steps rather than one very long run.  
This helps in case the virtual lab disconnects, since we can resume from checkpoints instead of starting over.


In [ ]:
# Example: go to epoch 2000 total
model, _ = train_to(2000)


### Checkpoint Resume
Before continuing training, check if a previous checkpoint exists.  
This allows training to pick up from where it left off instead of restarting from epoch 0.


In [ ]:
ckpt, start_epoch = latest_checkpoint()
print("latest ckpt:", ckpt, "| start_epoch:", start_epoch)


### Probe Training (10 Epochs)
Run a short 10-epoch training "probe" to verify that the model is loading correctly  
and that training updates are working before committing to a longer run.


In [ ]:
# ---- quick 10-epoch probe from epoch 500 ----
from tensorflow.keras.models import load_model

# load the checkpointed model (or build fresh if none)
model = load_model(ckpt) if ckpt else build_model(qmaze)

model = qtrain(
    model, qmaze,
    n_epoch_total=start_epoch + 10,   # go to 510 total
    start_epoch=start_epoch,
    max_memory=2000,
    batch_size=16,
    target_update_steps=500,
    checkpoint_every=50,              # save often during probe
    max_steps_per_episode=120,        # a bit smaller => faster
    log_every=1                       # print every epoch
)


### Continue Training (to 2000 Total Epochs)
Extend training from the probe run to reach a total of 2000 epochs.  
This balances training time with checkpoint safety and evaluation.


In [ ]:
# ---- continue training to 2000 total epochs ----
model = qtrain(
    model, qmaze,
    n_epoch_total=2000,               # absolute target
    start_epoch=start_epoch + 10,     # continue after the probe
    max_memory=2000,
    batch_size=16,
    target_update_steps=500,
    checkpoint_every=200,
    max_steps_per_episode=150,
    log_every=20                      # log every 20 epochs now
)


### Checkpoint Progress
Check the latest saved checkpoint after training to confirm progress and log the current epoch.


In [ ]:
ckpt, start_epoch = latest_checkpoint()
print("latest ckpt:", ckpt, "| start_epoch:", start_epoch)


### Additional Probe Run
If needed, reload the latest checkpoint and perform another short 10-epoch probe.  
This is useful to verify the model continues learning when resuming at later stages.


In [ ]:
from tensorflow.keras.models import load_model

ckpt, start_epoch = latest_checkpoint()
model = load_model(ckpt) if ckpt else build_model(qmaze)

model = qtrain(
    model, qmaze,
    n_epoch_total=start_epoch + 10,   # 500 -> 510
    start_epoch=start_epoch,
    max_memory=2000,
    batch_size=16,
    target_update_steps=500,
    checkpoint_every=50,              # save often
    max_steps_per_episode=120,        # a bit smaller -> faster
    log_every=1                       # print each epoch so we know it's moving
)


### Checkpoint Progress
Check the latest saved checkpoint after training to confirm progress and log the current epoch.


In [ ]:
ckpt, start_epoch = latest_checkpoint()
print("Now at checkpoint:", ckpt, "| start_epoch:", start_epoch)


### Resuming Training with Helper Function
Use the custom `resume_model()` helper to automatically reload the last checkpoint  
and continue training to the next target epoch (e.g., 600).


In [ ]:
model, start_epoch = resume_model()   # helper I gave earlier
model = qtrain(
    model, qmaze,
    n_epoch_total=600,    # next absolute target
    start_epoch=start_epoch,
    max_memory=2000,
    batch_size=16,
    target_update_steps=500,
    checkpoint_every=100,
    max_steps_per_episode=120,
    log_every=20
)


### Training to 5000 Total Epochs
After validating the first run (2000), scale training to 5000 epochs total.  
This staged approach avoids losing progress if the environment disconnects.


In [ ]:
# Next chunk: 5000 total
model, _ = train_to(5000)


### Final Training Target (15,000 Epochs)
Train the model to the full target of 15,000 epochs,  
as this typically allows the agent to converge to a stable and high win rate.


In [ ]:
# Final target: 15000 total
model, _ = train_to(15000)


### Model Evaluation and Demonstration
After training completes:
- Evaluate the model across 100 random start episodes to calculate average win rate.  
- Run a demo from a random start cell and from the fixed start `(0,0)` to show the learned path.  
- Perform a full completion check to confirm the agent can consistently reach the treasure.



### Model Evaluation and Demonstration
After training completes:
- Evaluate the model across 100 random start episodes to calculate average win rate.  
- Run a demo from a random start cell and from the fixed start `(0,0)` to show the learned path.  
- Perform a full completion check to confirm the agent can consistently reach the treasure.


In [ ]:
wr = evaluate(model, qmaze, n_episodes=100)
print("Final eval win-rate:", wr)

# Random start demo
start_demo = random.choice(qmaze.free_cells)
won, path = play_game(model, qmaze, pirate_cell=start_demo)
print("Random start:", start_demo, "Won?", won, "Path len:", len(path))
show(qmaze)

# Fixed start demo
won, path = play_game(model, qmaze, pirate_cell=(0,0))
print("Start (0,0) Won?", won, "Path len:", len(path))
show(qmaze)

# Completion check
print("Completion Check:", completion_check(model, qmaze))
